In [1]:
import pandas as pd
import numpy as np
import time
import os
import glob
import json
from collections import defaultdict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and test data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0005_19042021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

Model trained with data: 0005_19042021


## Model

In [3]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
print('Test -> X: {}, Y: {}'.format(X_test.shape, Y_test.shape))

results_ls = []

Train -> X: (943372, 15), Y: (943372, 24)
Test -> X: (161200, 15), Y: (161200, 24)


In [4]:
# Split data
train_samples = X_train.shape[0]
step = int(train_samples / 100000)
idx = list(range(0, train_samples, step))
X_train = X_train[idx, :]
Y_train = Y_train[idx, :]

print('Train -> X: {}, Y: {}'.format(X_train.shape, Y_train.shape))
print('Test -> X: {}, Y: {}'.format(X_test.shape, Y_test.shape))

Train -> X: (104820, 15), Y: (104820, 24)
Test -> X: (161200, 15), Y: (161200, 24)


In [9]:
# Test Lasso model
params = {
    'alpha': 0.0001
}

results = defaultdict(list)

tr_time = []
for target in range(Y_train.shape[1]):

    model = Lasso(**params, fit_intercept=True, random_state=0)
    
    t_start = time.time()
    model.fit(X_train, Y_train[:, target])
    tr_time.append(time.time() - t_start)
    
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    results['Train_MAE'].append(mean_absolute_error(Y_train[:, target], train_preds))
    results['Train_MSE'].append(mean_squared_error(Y_train[:, target], train_preds))
    results['Train_R2'].append(r2_score(Y_train[:, target], train_preds))
    results['Test_MAE'].append(mean_absolute_error(Y_test[:, target], test_preds))
    results['Test_MSE'].append(mean_squared_error(Y_test[:, target], test_preds))
    results['Test_R2'].append(r2_score(Y_test[:, target], test_preds))
    
print('Training time: {:.4f}'.format(sum(tr_time)))

results_summary = {}
results_summary['Training time'] = sum(tr_time)
# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results['_'.join([subset, loss])][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))
            results_summary[' '.join([subset, force, loss])] = '{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores))
            
results_ls.append(results_summary)

Training time: 42.6647
Train Fx MAE: 10.5679 ± 3.4259
Train Fx MSE: 227.7914 ± 150.4241
Train Fx R2: 0.2524 ± 0.0883
Train Fy MAE: 8.6873 ± 5.2562
Train Fy MSE: 224.8037 ± 254.7939
Train Fy R2: 0.1711 ± 0.0898
Train Fz MAE: 16.8071 ± 6.4160
Train Fz MSE: 571.2945 ± 501.8593
Train Fz R2: 0.2344 ± 0.1045
Test Fx MAE: 13.3392 ± 4.7360
Test Fx MSE: 377.4630 ± 253.5305
Test Fx R2: -0.1802 ± 0.2329
Test Fy MAE: 11.0378 ± 7.5093
Test Fy MSE: 384.7738 ± 478.5655
Test Fy R2: -0.1330 ± 0.1521
Test Fz MAE: 20.6163 ± 9.0945
Test Fz MSE: 1090.4435 ± 1262.4685
Test Fz R2: -0.4018 ± 0.5295
